In [1]:
from amplpy import add_to_path, AMPL

AMPL_DIR = r"C:\Users\melan\_MA 510\AMPL"  # Must match above
add_to_path(AMPL_DIR)
ampl = AMPL()
ampl.set_option('solver', 'path')
print(ampl.get_option('solver'))  # Should print 'path'

path


In [2]:
from amplpy import AMPL

ampl = AMPL()
ampl.set_option('solver', 'path')  # Must have PATH available

# Step 1: Declare sets
ampl.eval('''
set Aplays;
set Bplays;
''')

# Step 2: Assign set values in Python
ampl.set['Aplays'] = [1, 2]
ampl.set['Bplays'] = [1, 2]

# Step 3: Define the rest of the model
ampl.eval('''
param A_payoff {i in Aplays, j in Bplays};
param B_payoff {i in Aplays, j in Bplays};

var p {i in Aplays} >= 0;
var q {j in Bplays} >= 0;
var UA;
var UB;

s.t. A_sum: sum {i in Aplays} p[i] = 1;
s.t. B_sum: sum {j in Bplays} q[j] = 1;

s.t. A_payoff_ineq {i in Aplays}:
     sum {j in Bplays} A_payoff[i,j] * q[j] - UA <= 0;
s.t. A_comp {i in Aplays}:
     p[i] * ( sum {j in Bplays} A_payoff[i,j] * q[j] - UA ) = 0;

s.t. B_payoff_ineq {j in Bplays}:
     sum {i in Aplays} B_payoff[i,j] * p[i] - UB <= 0;
s.t. B_comp {j in Bplays}:
     q[j] * ( sum {i in Aplays} B_payoff[i,j] * p[i] - UB ) = 0;

option echo 0;
''')

# Step 4: Assign payoff matrices in Python
ampl.param['A_payoff'] = {(1,1):10, (1,2):3, (2,1):2, (2,2):9}
ampl.param['B_payoff'] = {(1,1):4, (1,2):6, (2,1):7, (2,2):5}

# === (a) General Nash Equilibrium ===
ampl.solve()
print("=== (a) Nash Equilibrium Solution ===")
print("A's mixed strategy (p):")
print(ampl.get_variable('p').get_values().to_pandas())
print("\nB's mixed strategy (q):")
print(ampl.get_variable('q').get_values().to_pandas())
print("\nA's expected payoff (UA):", ampl.get_variable('UA').value())
print("B's expected payoff (UB):", ampl.get_variable('UB').value())

# === (b) Fix A's strategy to standard 1 only, resolve ===
ampl.get_variable('p')[1].fix(1)
ampl.get_variable('p')[2].fix(0)
print("\n=== (b) Fix A's strategy: p[1]=1, p[2]=0 ===")
try:
    ampl.solve()
    print("A's strategy (p):")
    print(ampl.get_variable('p').get_values().to_pandas())
    print("\nB's strategy (q):")
    print(ampl.get_variable('q').get_values().to_pandas())
    print("\nA's expected payoff (UA):", ampl.get_variable('UA').value())
    print("B's expected payoff (UB):", ampl.get_variable('UB').value())
except Exception as e:
    print("No feasible solution found (as expected):", e)

Path 5.0.05: 8 side inequalities:
	4 simple bounds,
	0 range bounds,
	4 single inequality constraints, and
	0 range constraints.
Path 5.0.05 (Tue Jun 22 13:25:48 2021)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


INITIAL POINT STATISTICS
Zero row of order . . . . . . .  0.0000e+00 eqn: (_scon[1])
Zero row of order . . . . . . .  0.0000e+00 eqn: (_scon[2])
Zero row of order . . . . . . .  0.0000e+00 eqn: (_scon[3])
Zero row of order . . . . . . .  0.0000e+00 eqn: (_scon[4])
Total zero rows . . . . . . . .  4
Maximum of X. . . . . . . . . .  0.0000e+00 var: (_svar[1])
Maximum of F. . . . . . . . . .  1.0000e+00 eqn: (_scon[5])
Maximum of Grad F . . . . . . .  1.0000e+01 eqn: (_scon[7])
                                            var: (_svar[3])

INITIAL JACOBIAN NORM STATISTICS
Maximum Row Norm. . . . . . . .  1.5000e+01 eqn: (_scon[7])
Minimum Row Norm. . . . . . . .  2.0000e+00 eqn: (_scon[5])
Maximum Column Norm . . . . . .  1.4000e+01 var: (_svar[2])
Mini